In [1]:
import pandas as pd, json, re, os, pathlib, sys, matplotlib.pyplot as plt, numpy as np, random, collections, itertools
from matplotlib import rcParams
rcParams['font.family'] = "P052-Roman"
try:
    %load_ext autoreload
    %autoreload 2
    %config InlineBackend.figure_format = 'svg'
except SyntaxError:
    pass

os.chdir("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/")

In [2]:
OUR_RESULTS_FILE = "../../out/results_2023-02-21@18`51`09.5@+00`00.csv"
THEIR_RESULTS_FILE = "./41586_2022_5575_MOESM5_ESM.csv"

In [97]:
correctly_sort_index = lambda index: \
    [
        (x.split("|")[0],
        x.split("|")[1],
        int(x.split("|")[2][1:]))  #.str[2].tolist()] #str[1:].astype(int).tolist()
        for x in index
    ]

In [98]:
results = pd.read_csv(OUR_RESULTS_FILE)
results["KinSymbol"] = results["Kinase Gene Name"] + "|" + results["Kinase Uniprot Accession"]
results["SiteSymbol"] = (
    results["Site Gene Name"] + "|" + results["Site Uniprot Accession"] + "|" + results["Site Location"]
)
results = results[["KinSymbol", "SiteSymbol", "Score"]]
kin_to_site_to_score = results.set_index(["KinSymbol", "SiteSymbol"]).to_dict()["Score"]
cols = {x: {} for x in results["KinSymbol"].unique()}
for (kin, site), score in kin_to_site_to_score.items():
    cols[kin][site] = score
orig_keys = list(cols.keys())
for c in orig_keys:
    cols[c + "^Score"] = cols.pop(c)
derived_df = pd.DataFrame.from_dict(cols)
ranked = (
    derived_df.rank(axis=1, method="max")
    .rename(columns={c: str(c).replace("^Score", "^Rank") for c in derived_df.columns})
    .astype(int)
)
our_df_to_compare = pd.concat([derived_df, ranked], axis=1)
our_df_to_compare = our_df_to_compare[
    list(
        itertools.chain(
            *[
                [base_c, str(base_c).replace("^Score", "^Rank")]
                for base_c in our_df_to_compare.columns[: len(our_df_to_compare.columns) // 2]
            ]
        )
    )
]
our_df_to_compare = our_df_to_compare.sort_index(axis='columns')
our_ranks = our_df_to_compare[[x for x in our_df_to_compare.columns if "Rank" in str(x)]].copy()
our_ranks = our_ranks.sort_index(axis='index', key=correctly_sort_index, inplace=False)
our_ranks.to_csv("./our_ranks.csv")

In [99]:
their_results = pd.read_csv(THEIR_RESULTS_FILE).set_index("Uniprot Primary Accession")
matrix_name_to_uniprot_id: dict[str, str] = (
    pd.read_csv("./41586_2022_5575_MOESM3_ESM.csv").set_index("Matrix_name").to_dict()["Uniprot id"]
)
their_sites_to_symbols = [f"""{r['Gene'] if not pd.isna(r['Gene']) else f"<UNK>UnipAc:{i}"}|{i}|{r['Phosphosite']}""" for i, r in their_results.iterrows()]
their_kins_to_symbols = {
    tk: f"{re.sub(r'_rank', r'', tk)}|{matrix_name_to_uniprot_id[re.sub(r'_rank', r'', tk)]}^Rank"
    for tk in [str(x) for x in their_results.columns]
    if tk.endswith("_rank")
}
their_df_to_compare = their_results.rename(columns=their_kins_to_symbols)[list(their_kins_to_symbols.values())]
their_df_to_compare.index = pd.Index(their_sites_to_symbols)

their_ranks = their_df_to_compare[our_ranks.columns].copy()
their_ranks = their_ranks.rank(axis=1, method="max").astype(int)
their_ranks.sort_index(axis='index', inplace=True)
their_ranks.to_csv("./their_ranks.csv")

In [102]:
set(their_ranks.index) - set(our_ranks.index)

{'PDXDC1|Q6P996|S4',
 '<UNK>UnipAc:A0A0C4DFX4|A0A0C4DFX4|S2778',
 'TRAPPC2B|P0DI82|S119',
 'RANBP2|P49792|S2462',
 'ATP2A2|P16615|T357',
 'TUBA1C|Q9BQE3|S54',
 'SRGAP2B|P0DMP2|S206',
 'H2BC14|Q99879|S113',
 'FAM50A|Q14320|T164',
 'CEP170P1|Q96L14|T242',
 'TIA1|P31483|T83',
 'MYH1|P12882|S1780',
 'H2BC15|Q99877|S113',
 'H3-3A; H3-3B|P84243|S58',
 'GCOM2|Q9BZD3|S364',
 'H2BC4; H2BC6; H2BC7; H2BC8; H2BC10|P62807|S57',
 '<UNK>UnipAc:H7C4K7|H7C4K7|S56',
 'H2BC13|Q99880|S37',
 'WASHC2A|Q641Q2|S356',
 'ATF7-NPFF|K7ELQ4|S124',
 'LMNB1|P20700|S396',
 'DSP|P15924|S2833',
 'WASH4P|A8MWX3|S353',
 'PPAN-P2RY11|A0A0B4J1V8|S238',
 'MYH13|Q9UKX3|T451',
 '<UNK>UnipAc:A0A1B0GVI7|A0A1B0GVI7|T351',
 'RGPD2|P0DJD1|T889',
 'STK25|O00506|T174',
 'EEF1A1|P68104|S18',
 'RPS27A|P62979|T12',
 'ZNF658B|Q4V348|T777',
 'RTEL1-TNFRSF6B|F6WH68|S305',
 'MYH1|P12882|S1132',
 'TP63|Q9H3D4|S395',
 'UBQLN1|Q9UMX0|S260',
 'ACTG1|P63261|S365',
 'RGPD4|Q7Z3J3|S1232',
 'U2AF1L5|P0DN76|T11',
 'GSTM2|P28161|S27',
 'MYH2|Q9UKX2|

In [7]:
normalize_ranks = lambda obj_to_norm, starting_ranks, ending_ranks: obj_to_norm*(ending_ranks-1)/(starting_ranks-1) + (starting_ranks-ending_ranks)/(starting_ranks-1)